# Data wrangling WeRateDogs

## Table of Content
- [Introduction](#intro)
- [Gathering Data](#gather)
- [Accessing Data](#access)
    - [Quality](#qual)
    - [Tidiness](#tid)
- [Cleaning data](#clean)
- [Storing, Analyzing and Visualizing](#store)

<a id='intro'></a>
## Introduction
The WeRateDogs Twitter archive contains basic tweet data for all 5000+ of their tweets, but not everything. One column the archive does contain though: each tweet's text, which Udacity used to extract rating, dog name, and dog "stage" (i.e. doggo, floofer, pupper, and puppo) to make this Twitter archive "enhanced." Of the 5000+ tweets, Udacity have filtered for tweets with ratings only (there are 2356).

Udacity extracted this data programmatically, but We didn't do a very good job. The ratings probably aren't all correct. Same goes for the dog names and probably dog stages (see below for more information on these) too. You'll need to assess and clean these columns if you want to use them for analysis and visualization.

Our goal: wrangle WeRateDogs Twitter data to create interesting and trustworthy analyses and visualizations. The Twitter archive is great, but it only contains very basic tweet information. Additional gathering, then assessing and cleaning is required for "Wow!"-worthy analyses and visualizations.

<a id='gather'></a>
## Gathering Data
**1 .** <br>
The WeRateDogs Twitter archive. I am giving this file to you, so imagine it as a file on hand. Download this file manually by clicking the following link: twitter_archive_enhanced.csv

**2 .** <br> 
The tweet image predictions, i.e., what breed of dog (or other object, animal, etc.) is present in each tweet according to a neural network. This file (image_predictions.tsv) is hosted on Udacity's servers and should be downloaded programmatically using the Requests library and the following URL: https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv

**3 .** <br>
Each tweet's retweet count and favorite ("like") count at minimum, and any additional data you find interesting. Using the tweet IDs in the WeRateDogs Twitter archive, query the Twitter API for each tweet's JSON data using Python's Tweepy library and store each tweet's entire set of JSON data in a file called tweet_json.txt file. Each tweet's JSON data should be written to its own line. Then read this .txt file line by line into a pandas DataFrame with (at minimum) tweet ID, retweet count, and favorite count. 

**1. Reading Twitter archive file(twitter-archive-enhanced.csv)**

In [1]:
#importing necessary library's 
import pandas as pd
import numpy as np
import requests
import tweepy
from tweepy import OAuthHandler
import os
import time
import json
import re
%matplotlib inline

from datetime import datetime

In [2]:
#reading twitter-archive-enhanced.csv to a df
twitter_archive = pd.read_csv("twitter-archive-enhanced.csv")
twitter_archive.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


**This File include 2356 rows, 17 columns**
- tweet_id :- its unquiely identifies each tweets
- in_reply_to_status_id :- all the reply tweets with its status and orignal tweet ID's
- in_reply_to_user_id :- all the reply tweets with its integer representation of the original Tweet’s author ID and User_id might not always be mentioned.
- time_stamp :- Date & time when the tweet was sent
- source :- where tweet was posted
- text :- posted text in the tweet
- retweeted_status_id/retweeted_status_user_id/retweeted_status_timestamp :- all the status with tweet ID's or Tweet’s author ID or data & time of retweet.
- expanded_urls :- URL where the tweet is found
- rating_numerator :- The top number of a fraction
- ating_denominator :- The bottom number of a fraction
- name :- The name of the dog
- doggo / floofer / pupper / puppo :- Dog ages & stages

**2. Tweet image prediction (Request & Download)**

In [3]:
# Using the url given to access the site and request for download
url = "https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv"
request = requests.get(url)

In [4]:
# opening url request
with open(url.split('/')[-1], mode = 'wb') as file:
    file.write(request.content)

In [5]:
# Reading new requested file
images_predictions = pd.read_csv('image-predictions.tsv', sep = '\t')
images_predictions.head(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


**The file has 2,075 rows and 12 columns :**
- tweet_id:  its unquiely identifies each tweets
- jpg_URL: Where image is hosted
- img_num: Image number that corresponded to the most confident prediction
- p1: Algorithm's #1 predict for i.e. → golden retriever
- p1_conf: How confident the algorithm #1 prediction 
- p1_dog: Whether or not the #1 prediction is a breed of dog
- p2: Algorithm's second most likely prediction.
- p2_conf: How confident the algorithm  #2 prediction 
- p2_dog: Whether or not the #2 prediction is a breed of dog 

**3. Twitter API & JSON(not confirmation from twitter**

In [6]:
# load the data Tweeter API
# I did apply for Twitter developer account and it still in process

consumer_key = 'HIDDEN'
consumer_secret = 'HIDDEN'
access_token = 'HIDDEN'
access_secret = 'HIDDEN'

auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [7]:
# Use Twitter API to collect status data on tweets present in twitter1 dataframe
# Query Twitter's API for JSON data for each tweet ID in the Twitter archive

tweet_ids = list(twitter_archive['tweet_id'])

tweet_data = []
tweet_id_success = []
tweet_id_missing = []
for tweet_id in tweet_ids:
    try:
        data = api.get_status(tweet_id, 
                              tweet_mode='extended',
                              wait_on_rate_limit = True,
                              wait_on_rate_limit_notify = True)
        tweet_data.append(data)
        tweet_id_success.append(tweet_id)
    except:
        tweet_id_missing.append(tweet_id)
        print(tweet_id)

892420643555336193
892177421306343426
891815181378084864
891689557279858688
891327558926688256
891087950875897856
890971913173991426
890729181411237888
890609185150312448
890240255349198849
890006608113172480
889880896479866881
889665388333682689
889638837579907072
889531135344209921
889278841981685760
888917238123831296
888804989199671297
888554962724278272
888202515573088257
888078434458587136
887705289381826560
887517139158093824
887473957103951883
887343217045368832
887101392804085760
886983233522544640
886736880519319552
886680336477933568
886366144734445568
886267009285017600
886258384151887873
886054160059072513
885984800019947520
885528943205470208
885518971528720385
885311592912609280
885167619883638784
884925521741709313
884876753390489601
884562892145688576
884441805382717440
884247878851493888
884162670584377345
883838122936631299
883482846933004288
883360690899218434
883117836046086144
882992080364220416
882762694511734784
882627270321602560
882268110199369728
882045870035

820690176645140481
820494788566847489
820446719150292993
820314633777061888
820078625395449857
820013781606658049
819952236453363712
819924195358416896
819711362133872643
819588359383371776
819347104292290561
819238181065359361
819227688460238848
819015337530290176
819015331746349057
819006400881917954
819004803107983360
818646164899774465
818627210458333184
818614493328580609
818588835076603904
818536468981415936
818307523543449600
818259473185828864
818145370475810820
817908911860748288
817827839487737858
817777686764523521
817536400337801217
817502432452313088
817423860136083457
817415592588222464
817181837579653120
817171292965273600
817120970343411712
817056546584727552
816829038950027264
816816676327063552
816697700272001025
816450570814898180
816336735214911488
816091915477250048
816062466425819140
816014286006976512
815990720817401858
815966073409433600
815745968457060357
815736392542261248
815639385530101762
815390420867969024
814986499976527872
814638523311648768
814578408554

762035686371364864
761976711479193600
761750502866649088
761745352076779520
761672994376806400
761599872357261312
761371037149827077
761334018830917632
761292947749015552
761227390836215808
761004547850530816
760893934457552897
760656994973933572
760641137271070720
760539183865880579
760521673607086080
760290219849637889
760252756032651264
760190180481531904
760153949710192640
759943073749200896
759923798737051648
759846353224826880
759793422261743616
759566828574212096
759557299618865152
759447681597108224
759446261539934208
759197388317847553
759159934323924993
759099523532779520
759047813560868866
758854675097526272
758828659922702336
758740312047005698
758474966123810816
758467244762497024
758405701903519748
758355060040593408
758099635764359168
758041019896193024
757741869644341248
757729163776290825
757725642876129280
757611664640446465
757597904299253760
757596066325864448
757400162377592832
757393109802180609
757354760399941633
756998049151549440
756939218950160384
756651752796

707741517457260545
707738799544082433
707693576495472641
707629649552134146
707610948723478529
707420581654872064
707411934438625280
707387676719185920
707377100785885184
707315916783140866
707297311098011648
707059547140169728
707038192327901184
707021089608753152
707014260413456384
706904523814649856
706901761596989440
706681918348251136
706644897839910912
706593038911545345
706538006853918722
706516534877929472
706346369204748288
706310011488698368
706291001778950144
706265994973601792
706169069255446529
706166467411222528
706153300320784384
705975130514706432
705970349788291072
705898680587526145
705786532653883392
705591895322394625
705475953783398401
705442520700944385
705428427625635840
705239209544720384
705223444686888960
705102439679201280
705066031337840642
704871453724954624
704859558691414016
704847917308362754
704819833553219584
704761120771465216
704499785726889984
704491224099647488
704480331685040129
704364645503647744
704347321748819968
704134088924532736
704113298707

679844490799091713
679828447187857408
679777920601223168
679736210798047232
679729593985699840
679722016581222400
679530280114372609
679527802031484928
679511351870550016
679503373272485890
679475951516934144
679462823135686656
679405845277462528
679158373988876288
679148763231985668
679132435750195208
679111216690831360
679062614270468097
679047485189439488
679001094530465792
678991772295516161
678969228704284672
678800283649069056
678798276842360832
678774928607469569
678767140346941444
678764513869611008
678755239630127104
678740035362037760
678708137298427904
678675843183484930
678643457146150913
678446151570427904
678424312106393600
678410210315247616
678399652199309312
678396796259975168
678389028614488064
678380236862578688
678341075375947776
678334497360859136
678278586130948096
678255464182861824
678023323247357953
678021115718029313
677961670166224897
677918531514703872
677895101218201600
677716515794329600
677700003327029250
677698403548192770
677687604918272002
677673981332

669371483794317312
669367896104181761
669363888236994561
669359674819481600
669354382627049472
669353438988365824
669351434509529089
669328503091937280
669327207240699904
669324657376567296
669216679721873412
669214165781868544
669203728096960512
669037058363662336
669015743032369152
669006782128353280
669000397445533696
668994913074286592
668992363537309700
668989615043424256
668988183816871936
668986018524233728
668981893510119424
668979806671884288
668975677807423489
668967877119254528
668960084974809088
668955713004314625
668932921458302977
668902994700836864
668892474547511297
668872652652679168
668852170888998912
668826086256599040
668815180734689280
668779399630725120
668655139528511488
668645506898350081
668643542311546881
668641109086707712
668636665813057536
668633411083464705
668631377374486528
668627278264475648
668625577880875008
668623201287675904
668620235289837568
668614819948453888
668587383441514497
668567822092664832
668544745690562560
668542336805281792
668537837512

In [8]:
# Writing tweet data to json file
#with open('tweet_json.txt', mode = 'w') as file:
#    json.dump(tweet_data, file)

In [9]:
#Reading tweet_json.txt to df (twitter developer request is still pending)
tweet_json = pd.read_csv('tweet_json.txt', encoding = 'utf-8')
tweet_json.sample(5)

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
397,823719002937630720,0,12642,6629317,132918,2017-01-24 02:28:08
1490,692041934689402880,3651,1398,6629575,132918,2016-01-26 17:50:29
1948,673363615379013632,1064,332,6629688,132918,2015-12-06 04:49:31
1371,700847567345688576,2608,549,6629551,132918,2016-02-20 01:00:55
1575,686749460672679938,3745,1621,6629617,132918,2016-01-12 03:20:05


**The file has 2344 rows and 6 columns :**
- tweet_id:  its unquiely identifies each tweets
- favorites: Number of likes 
- retweets: Number of times this Tweet has been retweeted
- user_followers: Number of user followers
- user_favourites: Number of user likes
- date_time: Date & time when the tweet has been sent

In [10]:
# tweet_json df info()
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344 entries, 0 to 2343
Data columns (total 6 columns):
tweet_id           2344 non-null int64
favorites          2344 non-null int64
retweets           2344 non-null int64
user_followers     2344 non-null int64
user_favourites    2344 non-null int64
date_time          2344 non-null object
dtypes: int64(5), object(1)
memory usage: 110.0+ KB


<a id='access'></a>
# Accessing Data

In [11]:
# twitter_archive df 5 random sample
twitter_archive.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
634,793614319594401792,NaN,NaN,2016-11-02 00:42:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: When she says you're a good boy...,7.916723e+17,4.196984e+09,2016-10-27 16:06:04 +0000,https://twitter.com/dog_rates/status/791672322...,13,10,None,None,None,None,None
420,822163064745328640,NaN,NaN,2017-01-19 19:25:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Mattie. She's extremely...,7.862340e+17,4.196984e+09,2016-10-12 15:55:59 +0000,https://twitter.com/dog_rates/status/786233965...,11,10,Mattie,None,None,None,None
658,791672322847637504,NaN,NaN,2016-10-27 16:06:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When she says you're a good boy and you know y...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/791672322...,13,10,None,None,None,None,None
620,796125600683540480,NaN,NaN,2016-11-08 23:01:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",#ImWithThor 13/10\nhttps://t.co/a18mzkhTf6,NaN,NaN,NaN,https://twitter.com/king5seattle/status/796123...,13,10,None,None,None,None,None
953,751830394383790080,NaN,NaN,2016-07-09 17:28:29 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tucker. He's very camera shy. 12/10 wo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/751830394...,12,10,Tucker,None,None,None,None


In [12]:
# twitter_archive info
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [13]:
# twitter_archive description
twitter_archive.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [14]:
# twitter_archive all columns
twitter_archive.columns

Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name', 'doggo', 'floofer', 'pupper', 'puppo'],
      dtype='object')

In [15]:
# Column with missing value
twitter_archive.columns[twitter_archive.isnull().any()]

Index(['in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_timestamp',
       'expanded_urls'],
      dtype='object')

In [16]:
# twitter_archive df
twitter_archive.shape

(2356, 17)

In [17]:
# twitter_archive tweet_id
twitter_archive['tweet_id'].nunique()

2356

In [18]:
# twitter_archive tweet_id look
twitter_archive['tweet_id'].value_counts()

749075273010798592    1
741099773336379392    1
798644042770751489    1
825120256414846976    1
769212283578875904    1
700462010979500032    1
780858289093574656    1
699775878809702401    1
880095782870896641    1
760521673607086080    1
776477788987613185    1
691820333922455552    1
715696743237730304    1
714606013974974464    1
760539183865880579    1
813157409116065792    1
676430933382295552    1
743510151680958465    1
837012587749474308    1
833722901757046785    1
818259473185828864    1
670704688707301377    1
667160273090932737    1
674394782723014656    1
672082170312290304    1
670093938074779648    1
759923798737051648    1
809920764300447744    1
805487436403003392    1
838085839343206401    1
                     ..
763956972077010945    1
870308999962521604    1
720775346191278080    1
785927819176054784    1
783347506784731136    1
775733305207554048    1
834209720923721728    1
825026590719483904    1
758405701903519748    1
668986018524233728    1
6909388994772213

In [19]:
# proportion of null values in `in_reply_to_status_id`
len(twitter_archive[twitter_archive['in_reply_to_status_id'].isnull()])/len(twitter_archive['in_reply_to_status_id'])

# proportion of null values in `in_reply_to_user_id` as `in_reply_to_status_id`

0.966893039049236

In [20]:
# tweet timestamp value counts
twitter_archive['timestamp'].value_counts()

2016-01-03 01:39:57 +0000    1
2017-07-19 03:39:09 +0000    1
2016-08-27 00:47:53 +0000    1
2015-12-15 04:19:18 +0000    1
2015-12-11 17:51:04 +0000    1
2016-10-31 18:00:14 +0000    1
2016-06-11 01:13:51 +0000    1
2016-03-13 23:24:56 +0000    1
2016-06-17 16:01:16 +0000    1
2015-12-10 00:08:50 +0000    1
2015-12-01 03:39:03 +0000    1
2016-06-01 00:17:54 +0000    1
2016-01-24 03:14:07 +0000    1
2016-07-01 02:06:06 +0000    1
2017-05-10 00:08:34 +0000    1
2016-01-12 04:01:58 +0000    1
2016-03-03 19:32:29 +0000    1
2016-09-23 01:00:13 +0000    1
2017-01-15 21:49:15 +0000    1
2015-11-21 02:07:05 +0000    1
2015-11-30 03:06:07 +0000    1
2017-02-14 01:35:49 +0000    1
2017-06-11 21:18:31 +0000    1
2015-12-24 03:55:21 +0000    1
2017-02-06 17:02:17 +0000    1
2015-11-23 04:21:26 +0000    1
2016-01-16 15:40:14 +0000    1
2016-01-31 04:11:58 +0000    1
2015-12-09 18:59:46 +0000    1
2016-05-24 15:55:00 +0000    1
                            ..
2017-01-18 20:16:54 +0000    1
2016-12-

In [21]:
# explore time_stamp start to end date time
twitter_archive['timestamp'].min(), twitter_archive['timestamp'].max()

('2015-11-15 22:32:08 +0000', '2017-08-01 16:23:56 +0000')

In [22]:
# number of unique source from twitter_archive
twitter_archive['source'].nunique()

4

In [23]:
# Value of unique source from twitter_archive
twitter_archive['source'].value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [24]:
# lets look at the test in twitter_archive
twitter_archive['text'].value_counts()

This is Timofy. He's a pilot for Southwest. It's Christmas morning &amp; everyone has gotten kickass gifts but him. 9/10 https://t.co/3FuNbzyPwo                          1
This is Maxaroni. He's curly af. Also rather fabulous. 11/10 would hug well https://t.co/A216OjIdca                                                                       1
This is Bo. He was a very good First Doggo. 14/10 would be an absolute honor to pet https://t.co/AdPKrI8BZ1                                                               1
This is Pickles. She's a silly pupper. Thinks she's a dish. 12/10 would dry https://t.co/7mPCF4ZwEk                                                                       1
This is Harper. She scraped her elbow attempting a backflip off a tree. Valiant effort tho. 12/10 https://t.co/oHKJHghrp5                                                 1
This is Pavlov. His floatation device has failed him. He's quite pupset about it. 11/10 would rescue https://t.co/MXd0AGDsRJ                

In [25]:
# retweeted_status_id unique count
twitter_archive['retweeted_status_id'].nunique()

181

In [26]:
# retweeted_status_id value counts
twitter_archive['retweeted_status_id'].value_counts()

7.757333e+17    1
7.507196e+17    1
6.742918e+17    1
6.833919e+17    1
8.269587e+17    1
8.780576e+17    1
7.320056e+17    1
7.186315e+17    1
6.732953e+17    1
7.914070e+17    1
8.447048e+17    1
7.862340e+17    1
8.685523e+17    1
7.504293e+17    1
8.327664e+17    1
6.690004e+17    1
6.873173e+17    1
7.638376e+17    1
7.815247e+17    1
8.092201e+17    1
8.000650e+17    1
8.174239e+17    1
8.001414e+17    1
7.909461e+17    1
7.867091e+17    1
8.406323e+17    1
8.688804e+17    1
7.869631e+17    1
7.733088e+17    1
8.222448e+17    1
               ..
6.816941e+17    1
7.899865e+17    1
7.939622e+17    1
6.800555e+17    1
7.128090e+17    1
8.164506e+17    1
6.769365e+17    1
6.675487e+17    1
8.083449e+17    1
7.626999e+17    1
6.678667e+17    1
8.479710e+17    1
6.820881e+17    1
8.352641e+17    1
7.761133e+17    1
7.902771e+17    1
6.675484e+17    1
7.677549e+17    1
8.782815e+17    1
8.663350e+17    1
7.399792e+17    1
7.403732e+17    1
8.395493e+17    1
7.001438e+17    1
8.482894e+

In [27]:
# Retweeted_timestamp value counts
twitter_archive['retweeted_status_timestamp'].value_counts()

2016-09-01 16:14:48 +0000    1
2017-02-16 23:23:38 +0000    1
2017-03-08 01:41:24 +0000    1
2016-11-22 20:58:07 +0000    1
2016-02-21 01:19:47 +0000    1
2016-07-13 01:34:21 +0000    1
2016-07-27 00:40:12 +0000    1
2017-06-09 16:22:42 +0000    1
2016-08-01 16:42:51 +0000    1
2016-06-08 02:41:38 +0000    1
2017-04-01 21:42:03 +0000    1
2015-12-02 03:40:57 +0000    1
2017-01-13 17:00:21 +0000    1
2016-08-22 16:06:54 +0000    1
2016-08-01 01:28:46 +0000    1
2016-07-30 17:56:51 +0000    1
2016-03-24 01:11:29 +0000    1
2015-12-22 04:35:49 +0000    1
2016-04-29 00:21:01 +0000    1
2016-10-14 16:13:10 +0000    1
2017-02-18 01:39:12 +0000    1
2016-01-13 16:56:30 +0000    1
2016-12-09 06:17:20 +0000    1
2016-10-23 19:42:02 +0000    1
2017-01-02 18:38:42 +0000    1
2015-12-12 02:23:01 +0000    1
2016-10-21 18:16:44 +0000    1
2015-11-19 00:32:12 +0000    1
2015-12-24 16:00:30 +0000    1
2017-06-19 17:14:49 +0000    1
                            ..
2015-11-28 03:31:48 +0000    1
2016-08-

In [28]:
# looking at urls in twitter_archive
twitter_archive['expanded_urls'][:5]

0    https://twitter.com/dog_rates/status/892420643...
1    https://twitter.com/dog_rates/status/892177421...
2    https://twitter.com/dog_rates/status/891815181...
3    https://twitter.com/dog_rates/status/891689557...
4    https://twitter.com/dog_rates/status/891327558...
Name: expanded_urls, dtype: object

In [29]:
twitter_archive['expanded_urls'].value_counts()[:10]

https://twitter.com/dog_rates/status/829501995190984704/photo/1,https://twitter.com/dog_rates/status/829501995190984704/photo/1                                                                                                                                    2
https://twitter.com/dog_rates/status/773308824254029826/photo/1                                                                                                                                                                                                    2
https://twitter.com/dog_rates/status/791406955684368384/photo/1,https://twitter.com/dog_rates/status/791406955684368384/photo/1,https://twitter.com/dog_rates/status/791406955684368384/photo/1,https://twitter.com/dog_rates/status/791406955684368384/photo/1    2
https://twitter.com/dog_rates/status/816450570814898180/photo/1,https://twitter.com/dog_rates/status/816450570814898180/photo/1                                                                                          

In [30]:
# rating denominator
twitter_archive['rating_denominator'].value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

In [31]:
# rating numerator
twitter_archive['rating_numerator'].value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

In [32]:
# rating_numerator is distributed across 0- max(1776)
# getting rating_numerator greater than 20.
twitter_archive.query("rating_numerator > 20")

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
188,855862651834028034,8.558616e+17,1.943518e+08,2017-04-22 19:15:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@dhmontgomery We also gave snoop dogg a 420/10...,NaN,NaN,NaN,NaN,420,10,None,None,None,None,None
189,855860136149123072,8.558585e+17,1.361572e+07,2017-04-22 19:05:32 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@s8n You tried very hard to portray this good ...,NaN,NaN,NaN,NaN,666,10,None,None,None,None,None
290,838150277551247360,8.381455e+17,2.195506e+07,2017-03-04 22:12:52 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@markhoppus 182/10,NaN,NaN,NaN,NaN,182,10,None,None,None,None,None
313,835246439529840640,8.352460e+17,2.625958e+07,2017-02-24 21:54:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@jonnysun @Lin_Manuel ok jomny I know you're e...,NaN,NaN,NaN,NaN,960,0,None,None,None,None,None
340,832215909146226688,NaN,NaN,2017-02-16 13:11:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: This is Logan, the Chow who liv...",7.867091e+17,4.196984e+09,2016-10-13 23:23:56 +0000,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
433,820690176645140481,NaN,NaN,2017-01-15 17:52:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",The floofs have been released I repeat the flo...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/820690176...,84,70,None,None,None,None,None
516,810984652412424192,NaN,NaN,2016-12-19 23:06:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Sam. She smiles 24/7 &amp; secretly aspir...,NaN,NaN,NaN,"https://www.gofundme.com/sams-smile,https://tw...",24,7,Sam,None,None,None,None
695,786709082849828864,NaN,NaN,2016-10-13 23:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Logan, the Chow who lived. He solemnly...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/786709082...,75,10,Logan,None,None,None,None
763,778027034220126208,NaN,NaN,2016-09-20 00:24:34 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Sophie. She's a Jubilant Bush Pupper. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/778027034...,27,10,Sophie,None,None,pupper,None
902,758467244762497024,NaN,NaN,2016-07-28 01:00:57 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Why does this never happen at my front door......,NaN,NaN,NaN,https://twitter.com/dog_rates/status/758467244...,165,150,None,None,None,None,None


In [33]:
# unique name in twitter_archive
twitter_archive['name'].nunique()

957

In [34]:
# unqiue name count in twitter_archive
twitter_archive['name'].value_counts()

None         745
a             55
Charlie       12
Lucy          11
Cooper        11
Oliver        11
Lola          10
Tucker        10
Penny         10
Winston        9
Bo             9
Sadie          8
the            8
Daisy          7
an             7
Bailey         7
Buddy          7
Toby           7
Scout          6
Koda           6
Dave           6
Stanley        6
Milo           6
Jack           6
Leo            6
Rusty          6
Oscar          6
Jax            6
Bella          6
Oakley         5
            ... 
Farfle         1
Huxley         1
Yukon          1
Mosby          1
Filup          1
Philippe       1
Monster        1
Luther         1
Brian          1
Kara           1
Dook           1
Hall           1
Rooney         1
Kaiya          1
Kevon          1
Staniel        1
Divine         1
Ace            1
Zeus           1
Karl           1
Mookie         1
Jebberson      1
Kuyu           1
old            1
Alfy           1
Cuddles        1
Darla          1
Kingsley      

In [35]:
# when looking at name in twitter_archive we found out that there were
# few invalid names like null or a or two letter words 
# as basically most name have atleast  letters
twitter_archive[twitter_archive['name'].apply(len) < 3]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
56,881536004380872706,NaN,NaN,2017-07-02 15:32:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a pupper approaching maximum borkdrive...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/881536004...,14,10,a,None,None,pupper,None
393,825876512159186944,NaN,NaN,2017-01-30 01:21:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Mo. No one will push him around in the...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/825876512...,11,10,Mo,None,None,None,None
446,819015337530290176,NaN,NaN,2017-01-11 02:57:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo. He was a very good ...,8.190048e+17,4.196984e+09,2017-01-11 02:15:36 +0000,https://twitter.com/dog_rates/status/819004803...,14,10,Bo,doggo,None,None,None
449,819004803107983360,NaN,NaN,2017-01-11 02:15:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He was a very good First Doggo. 14...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/819004803...,14,10,Bo,doggo,None,None,None
553,804026241225523202,NaN,NaN,2016-11-30 18:16:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He's going to make me cry. 13/10 p...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/804026241...,13,10,Bo,None,None,None,None
583,800188575492947969,NaN,NaN,2016-11-20 04:06:37 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo. He's a Benedoop Cum...,6.816941e+17,4.196984e+09,2015-12-29 04:31:49 +0000,https://twitter.com/dog_rates/status/681694085...,11,10,Bo,None,None,pupper,None
649,792913359805018113,NaN,NaN,2016-10-31 02:17:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here is a perfect example of someone who has t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/792913359...,13,10,a,None,None,None,None
679,789137962068021249,NaN,NaN,2016-10-20 16:15:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bo. He's a West Congolese Bugaboop Snu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/789137962...,12,10,Bo,None,None,None,None
686,788070120937619456,NaN,NaN,2016-10-17 17:32:13 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Bo and Ty. Bo eats pape...,7.610045e+17,4.196984e+09,2016-08-04 01:03:17 +0000,https://twitter.com/dog_rates/status/761004547...,11,10,Bo,None,None,None,None
759,778396591732486144,NaN,NaN,2016-09-21 00:53:04 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is an East African Chalupa...,7.030419e+17,4.196984e+09,2016-02-26 02:20:37 +0000,https://twitter.com/dog_rates/status/703041949...,10,10,an,None,None,None,None


In [36]:
# propotition of 'in_reply_to_status_id' which is null
len(twitter_archive[twitter_archive['in_reply_to_status_id'].isnull()])/twitter_archive.shape[0]

0.966893039049236

In [37]:
# propotition of 'in_reply_to_status_id' which is null
len(twitter_archive[twitter_archive['in_reply_to_user_id'].isnull()])/twitter_archive.shape[0]

0.966893039049236

In [38]:
#looking at images_predition dataset
images_predictions.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
322,671866342182637568,https://pbs.twimg.com/media/CVLy3zFWoAA93qJ.jpg,1,Labrador_retriever,0.875614,True,Chihuahua,0.032182,True,golden_retriever,1.723250e-02,True
655,682242692827447297,https://pbs.twimg.com/media/CXfQG_fW8AAjVhV.jpg,1,snorkel,0.504983,False,loggerhead,0.345298,False,scuba_diver,7.475390e-02,False
522,676582956622721024,https://pbs.twimg.com/media/CWO0m8tUwAAB901.jpg,1,seat_belt,0.790028,False,Boston_bull,0.196307,True,French_bulldog,1.242890e-02,True
149,668636665813057536,https://pbs.twimg.com/media/CUd5gBGWwAA0IVA.jpg,1,komondor,0.999956,True,llama,0.000043,False,ram,2.160900e-07,False
77,667437278097252352,https://pbs.twimg.com/media/CUM2qWaWoAUZ06L.jpg,1,porcupine,0.989154,False,bath_towel,0.006300,False,badger,9.663400e-04,False


In [39]:
#looking at images_predictions columns
images_predictions.columns

Index(['tweet_id', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog', 'p2',
       'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'],
      dtype='object')

In [40]:
images_predictions.columns[images_predictions.isnull().any()]

Index([], dtype='object')

In [41]:
#images_predictions info
images_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [42]:
# jpg_url duplicate
images_predictions[images_predictions['jpg_url'].duplicated(keep='first')]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1315,754874841593970688,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,pug,0.272205,True,bull_mastiff,0.251530,True,bath_towel,0.116806,False
1333,757729163776290825,https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg,2,cash_machine,0.802333,False,schipperke,0.045519,True,German_shepherd,0.023353,True
1345,759159934323924993,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True
1349,759566828574212096,https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg,1,Labrador_retriever,0.967397,True,golden_retriever,0.016641,True,ice_bear,0.014858,False
1364,761371037149827077,https://pbs.twimg.com/tweet_video_thumb/CeBym7...,1,brown_bear,0.713293,False,Indian_elephant,0.172844,False,water_buffalo,0.038902,False
1368,761750502866649088,https://pbs.twimg.com/media/CYLDikFWEAAIy1y.jpg,1,golden_retriever,0.586937,True,Labrador_retriever,0.398260,True,kuvasz,0.005410,True
1387,766078092750233600,https://pbs.twimg.com/media/ChK1tdBWwAQ1flD.jpg,1,toy_poodle,0.420463,True,miniature_poodle,0.132640,True,Chesapeake_Bay_retriever,0.121523,True
1407,770093767776997377,https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg,1,golden_retriever,0.843799,True,Labrador_retriever,0.052956,True,kelpie,0.035711,True
1417,771171053431250945,https://pbs.twimg.com/media/CVgdFjNWEAAxmbq.jpg,3,Samoyed,0.978833,True,Pomeranian,0.012763,True,Eskimo_dog,0.001853,True


In [43]:
#img_num value count
images_predictions['img_num'].value_counts()

1    1780
2     198
3      66
4      31
Name: img_num, dtype: int64

In [44]:
#looking at breed at prediction 1
images_predictions['p1'].nunique()

378

In [45]:
# Most common breed on prediction 1
images_predictions['p1'].value_counts()[:10]

golden_retriever      150
Labrador_retriever    100
Pembroke               89
Chihuahua              83
pug                    57
chow                   44
Samoyed                43
toy_poodle             39
Pomeranian             38
malamute               30
Name: p1, dtype: int64

In [46]:
# looking at tweet_json data sampling 5 
tweet_json.sample(5)

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
1483,692535307825213440,3436,1497,6629573,132918,2016-01-28 02:30:58
1237,711306686208872448,3536,803,6629532,132918,2016-03-19 21:41:44
314,834167344700198914,16933,4028,6629306,132918,2017-02-21 22:26:07
1896,674436901579923456,1170,422,6629683,132918,2015-12-09 03:54:22
398,823699002998870016,13639,2709,6629317,132918,2017-01-24 01:08:40


In [47]:
#tweet_json info
tweet_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2344 entries, 0 to 2343
Data columns (total 6 columns):
tweet_id           2344 non-null int64
favorites          2344 non-null int64
retweets           2344 non-null int64
user_followers     2344 non-null int64
user_favourites    2344 non-null int64
date_time          2344 non-null object
dtypes: int64(5), object(1)
memory usage: 110.0+ KB


In [48]:
#top 10 account having largest followers 
tweet_json.nlargest(10, ['user_followers'])

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
2337,666051853826850816,1231,859,6629803,132918,2015-11-16 00:35:11
2338,666050758794694657,133,58,6629803,132918,2015-11-16 00:30:50
2339,666049248165822465,109,40,6629803,132918,2015-11-16 00:24:50
2340,666044226329800704,301,141,6629803,132918,2015-11-16 00:04:52
2341,666033412701032449,125,44,6629803,132918,2015-11-15 23:21:54
2342,666029285002620928,131,47,6629803,132918,2015-11-15 23:05:30
2343,666020888022790149,2561,520,6629803,132918,2015-11-15 22:32:08
2332,666071193221509120,148,62,6629802,132918,2015-11-16 01:52:02
2333,666063827256086533,480,223,6629802,132918,2015-11-16 01:22:45
2334,666058600524156928,113,58,6629802,132918,2015-11-16 01:01:59


In [49]:
# maximum retweets 
tweet_json.nlargest(10, ['retweets'])

,tweet_id,favorites,retweets,user_followers,user_favourites,date_time
1028,744234799360020481,128704,77689,6629469,132918,2016-06-18 18:26:18
525,807106840509214720,123429,61405,6629327,132918,2016-12-09 06:17:20
807,770743923962707968,0,51123,6629357,132918,2016-08-30 22:04:05
1068,739238157791694849,73632,51123,6629474,132918,2016-06-04 23:31:25
404,822872901745569793,143885,49372,6629319,132918,2017-01-21 18:26:02
65,879415818425184262,106309,44810,6629278,132918,2017-06-26 19:07:24
437,819015337530290176,0,41172,6629321,132918,2017-01-11 02:57:27
440,819004803107983360,93982,41172,6629321,132918,2017-01-11 02:15:36
528,806629075125202948,82509,39649,6629327,132918,2016-12-07 22:38:52
131,866450705531457537,124657,36663,6629288,132918,2017-05-22 00:28:40


In [50]:
tweet_json.shape

(2344, 6)

In [51]:
tweet_json.columns[tweet_json.isnull().any()]

Index([], dtype='object')

In [52]:
tweet_json['tweet_id'].value_counts()

749075273010798592    1
741099773336379392    1
798644042770751489    1
825120256414846976    1
769212283578875904    1
700462010979500032    1
780858289093574656    1
699775878809702401    1
880095782870896641    1
760521673607086080    1
776477788987613185    1
691820333922455552    1
715696743237730304    1
714606013974974464    1
760539183865880579    1
813157409116065792    1
676430933382295552    1
743510151680958465    1
833722901757046785    1
805487436403003392    1
759923798737051648    1
674742531037511680    1
670704688707301377    1
667160273090932737    1
674394782723014656    1
672082170312290304    1
670093938074779648    1
818259473185828864    1
842765311967449089    1
809920764300447744    1
                     ..
870308999962521604    1
720775346191278080    1
879492040517615616    1
775733305207554048    1
667911425562669056    1
834209720923721728    1
825026590719483904    1
758405701903519748    1
668986018524233728    1
690938899477221376    1
6788002836490690

## Assess : ##

This section will go over the following:
- Quality Issues
- Tidiness Issues
<a id='qual'></a>

### Quality Issues ###

### Twitter_Archive: ###
- there are missing value in following columns : 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_id','retweeted_status_user_id', 'retweeted_status_timestamp','expanded_urls'
- name with 745 missing value as None(not Nan)
- some name are invalid or false as just 'a'
- some expanded_urls tweets or retweets are without images
- tweet_id is int64 type as it should be object type as no calucation is needed
- timestamp and retweet_status_timestamp is also object type
- having retweets might bring duplicates.
- max rating_numerator is 1776


### Images_Prediction: ###
- tweet_id is int64 type as it should be object type as no calucation is needed
- p1,p2,p3 columns dog breeds are not consistently with case sensitive(lower or uppercase)
- 66 jpg_url duplicates 
- Delete useless columns

### tweet_data(JSON) : ###
- tweet_id is int64 type as it should be object type as no calucation is needed

<a id='tid'></a>
### Tidiness Issues : ###


_Issues_ :
- Twitter_Archive, Images_Predictions, Tweet_json : Join all three dataset into one main dataset
- Twitter_Archive : contains four column(dogger, floofer, pupper and puppo) needs to make it into one
- Twitter_Archive : All Prediction columns must be packed into one single columns
- Twitter_Archive : packing applies to Prediction_confidence too
- Change tweet_id int64 type to object type

In [53]:
# creating copy of dataset
twitter_1 = tweet_json.copy()
twitter_2 = twitter_archive.copy()
images_clean = images_predictions.copy()

<a id='clean'></a>
## Cleaning Data: ##
## _Merge all dataframes_ ##
### _Define:_ ###
- adding tweet_data and image to twitter_archive table.

### _Code:_ ###

In [54]:
# Merging datasets 
twitter_archive_clean = pd.merge(left=twitter_2,
                                 right=twitter_1, left_on='tweet_id', right_on='tweet_id', how='inner')

In [55]:
twitter_archive_clean = twitter_archive_clean.merge(images_clean, on='tweet_id', how='inner')

## **Test** ##

In [56]:
# looking into info
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2068 entries, 0 to 2067
Data columns (total 33 columns):
tweet_id                      2068 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     2068 non-null object
source                        2068 non-null object
text                          2068 non-null object
retweeted_status_id           75 non-null float64
retweeted_status_user_id      75 non-null float64
retweeted_status_timestamp    75 non-null object
expanded_urls                 2068 non-null object
rating_numerator              2068 non-null int64
rating_denominator            2068 non-null int64
name                          2068 non-null object
doggo                         2068 non-null object
floofer                       2068 non-null object
pupper                        2068 non-null object
puppo                         2068 non-null object
favorites                     2068 

## _Define_ : ##
- all this columns 'doggo', 'floofer', 'pupper' and 'puppo' melts into one column 'dog_stage'.

## _Code_ : ##

In [57]:
#looking at the four columns
twitter_archive_clean.groupby(["doggo", "floofer", "pupper", "puppo"]).size().reset_index().rename(columns={0: "count"})

,doggo,floofer,pupper,puppo,count
0,None,None,None,None,1747
1,None,None,None,puppo,23
2,None,None,pupper,None,211
3,None,floofer,None,None,7
4,doggo,None,None,None,67
5,doggo,None,None,puppo,1
6,doggo,None,pupper,None,11
7,doggo,floofer,None,None,1


In [58]:
# replacing all None to ""
twitter_archive_clean.doggo.replace('None', '', inplace=True)
twitter_archive_clean.floofer.replace('None', '', inplace=True)
twitter_archive_clean.pupper.replace('None', '', inplace=True)
twitter_archive_clean.puppo.replace('None', '', inplace=True)

In [59]:
#replaining all Nan to ""
twitter_archive_clean.doggo.replace(np.NaN, '', inplace=True)
twitter_archive_clean.floofer.replace(np.NaN, '', inplace=True)
twitter_archive_clean.pupper.replace(np.NaN, '', inplace=True)
twitter_archive_clean.puppo.replace(np.NaN, '', inplace=True)

In [62]:
# Creating stage column and since Doggo is biggest stages among all so i have all dogs with two stages to doggo
twitter_archive_clean['dog_stage'] = twitter_archive_clean.doggo + twitter_archive_clean.floofer + twitter_archive_clean.pupper + twitter_archive_clean.puppo
twitter_archive_clean.loc[twitter_archive_clean.dog_stage == 'doggopupper', 'dog_stage'] = 'doggo'
twitter_archive_clean.loc[twitter_archive_clean.dog_stage == 'doggopuppo', 'dog_stage'] = 'doggo'
twitter_archive_clean.loc[twitter_archive_clean.dog_stage == 'doggofloofer', 'dog_stage'] = 'doggo'

## **Test**##

In [65]:
# dog_stage value_counts to different stages
print(twitter_archive_clean.dog_stage.value_counts())
# total stages
print(len(twitter_archive_clean))

           1747
pupper      211
doggo        80
puppo        23
floofer       7
Name: dog_stage, dtype: int64
2068


## _Define_ : ##
- replace name that are likely not real such as 'a', 'an'

## _Code_ : ##

In [66]:
# Remove all invalid name
dogsname = ['None', 'a', 'the', 'an', 'this', 'such', 'quite', 'not', 'one', 'incredibly', 'mad', 'very', 'just', 'my']

for word in dogsname:
    twitter_archive_clean.loc[twitter_archive_clean.name == word, 'name'] = None

## Test

In [67]:
# printing name value_counts
twitter_archive_clean.name.value_counts()

Charlie       11
Penny         10
Tucker        10
Cooper        10
Lucy          10
Oliver        10
Lola           8
Winston        8
Bo             8
Sadie          8
Toby           7
Daisy          7
Stanley        6
Jax            6
Dave           6
Bailey         6
Rusty          6
Koda           6
Bella          6
Scout          6
Milo           6
Alfie          5
Buddy          5
Larry          5
Louis          5
Leo            5
Chester        5
Oscar          5
Dexter         4
Ruby           4
              ..
Aubie          1
Cermet         1
Blakely        1
William        1
Ben            1
Chesterson     1
Daniel         1
Henry          1
Stefan         1
Charl          1
Finnegus       1
Eugene         1
Burt           1
Hamrick        1
Randall        1
Craig          1
Brandonald     1
Ralpher        1
Jeffrie        1
Carly          1
Lilli          1
Mauve          1
Jo             1
Bradley        1
Mitch          1
Dante          1
Simba          1
Harrison      

## _Define_ : ##
- Creating Breed column using predictions

## _Code_ : ##

In [68]:
# Creating Breed column using P1, P2, P3 predictions
breed = []
confidence= []

def breed_type(row):
    if row['p1_dog']:
        breed.append(row['p1'])
        confidence.append(row['p1_conf'])
    elif row['p2_dog']:
        breed.append(row['p2'])
        confidence.append(row['p2_conf'])
    elif row['p3_dog']:
        breed.append(row['p3'])
        confidence.append(row['p3_conf'])
    else:
        breed.append('Unidentifiable')
        confidence.append(0)

twitter_archive_clean.apply(breed_type, axis = 1)
# adding bread and confidence along
twitter_archive_clean['breed'] = breed
twitter_archive_clean['confidence'] = confidence

## _Define_ : ##
- Clean rows and columns that we dont need

## _Code_ : ##

In [69]:
# Delete the retweets_status_id null
twitter_archive_clean = twitter_archive_clean[pd.isnull(twitter_archive_clean.retweeted_status_id)]

# Delete duplicated tweet_id
twitter_archive_clean = twitter_archive_clean.drop_duplicates()

# Delete tweets with no pictures
twitter_archive_clean = twitter_archive_clean.dropna(subset = ['jpg_url'])

# small test
len(twitter_archive_clean)

1993

In [70]:
# Checking info of data (changes)
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 2067
Data columns (total 36 columns):
tweet_id                      1993 non-null int64
in_reply_to_status_id         23 non-null float64
in_reply_to_user_id           23 non-null float64
timestamp                     1993 non-null object
source                        1993 non-null object
text                          1993 non-null object
retweeted_status_id           0 non-null float64
retweeted_status_user_id      0 non-null float64
retweeted_status_timestamp    0 non-null object
expanded_urls                 1993 non-null object
rating_numerator              1993 non-null int64
rating_denominator            1993 non-null int64
name                          1361 non-null object
doggo                         1993 non-null object
floofer                       1993 non-null object
pupper                        1993 non-null object
puppo                         1993 non-null object
favorites                     1993 non

In [71]:
# delete not useful columns
twitter_archive_clean.drop(['in_reply_to_status_id','in_reply_to_user_id','retweeted_status_id','retweeted_status_user_id','retweeted_status_timestamp','p1_conf','p1_dog', 'p2_conf','p2_dog', 'p3_conf','p3_dog'], axis=1, inplace=True)

In [72]:
# Delete column date_time 
twitter_archive_clean = twitter_archive_clean.drop('date_time', 1)

# small test
list(twitter_archive_clean)

['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floofer',
 'pupper',
 'puppo',
 'favorites',
 'retweets',
 'user_followers',
 'user_favourites',
 'jpg_url',
 'img_num',
 'p1',
 'p2',
 'p3',
 'dog_stage',
 'breed',
 'confidence']

In [73]:
#Delete dog_stage duplicates
twitter_archive_clean = twitter_archive_clean.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')

## **Test** ##

In [74]:
print(twitter_archive_clean.dog_stage.value_counts())
print(len(twitter_archive_clean))

           1687
pupper      203
doggo        74
puppo        22
floofer       7
Name: dog_stage, dtype: int64
1993


## Define :
- Clean the source in dataset

## Code :


In [75]:
# Clean the content of source column
twitter_archive_clean['source'] = twitter_archive_clean['source'].apply(lambda x: re.findall(r'>(.*)<', x)[0])

In [76]:
# Test
twitter_archive_clean

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,...,user_followers,user_favourites,jpg_url,img_num,p1,p2,p3,dog_stage,breed,confidence
0,892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,,,...,6629272,132918,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1,orange,bagel,banana,,Unidentifiable,0.000000
1382,683857920510050305,2016-01-04 03:50:08 +0000,Twitter for iPhone,Meet Sadie. She fell asleep on the beach and h...,https://twitter.com/dog_rates/status/683857920...,10,10,Sadie,,,...,6629636,132918,https://pbs.twimg.com/media/CX2NJmRWYAAxz_5.jpg,1,bluetick,Shetland_sheepdog,beagle,,bluetick,0.174738
1381,684097758874210310,2016-01-04 19:43:10 +0000,Twitter for iPhone,Say hello to Lupe. This is how she sleeps. 10/...,https://twitter.com/dog_rates/status/684097758...,10,10,Lupe,,,...,6629636,132918,https://pbs.twimg.com/media/CX5nR5oWsAAiclh.jpg,1,Labrador_retriever,German_short-haired_pointer,Chesapeake_Bay_retriever,,Labrador_retriever,0.627856
1380,684122891630342144,2016-01-04 21:23:02 +0000,Twitter for iPhone,Heartwarming scene of two pups that want nothi...,https://twitter.com/dog_rates/status/684122891...,11,10,None,,,...,6629635,132918,https://pbs.twimg.com/media/CX5-HslWQAIiXKB.jpg,1,cheetah,Arabian_camel,jaguar,,Unidentifiable,0.000000
1379,684177701129875456,2016-01-05 01:00:50 +0000,Twitter for iPhone,This is Kulet. She's very proud of the flower ...,https://twitter.com/dog_rates/status/684177701...,10,10,Kulet,,,...,6629635,132918,https://pbs.twimg.com/media/CX6v_JOWsAE0beZ.jpg,1,chow,German_shepherd,golden_retriever,,chow,0.334783
1378,684188786104872960,2016-01-05 01:44:52 +0000,Twitter for iPhone,"""Yo Boomer I'm taking a selfie, grab your stic...",https://twitter.com/dog_rates/status/684188786...,10,10,None,,,...,6629633,132918,https://pbs.twimg.com/media/CX66EiJWkAAVjA-.jpg,1,kelpie,American_Staffordshire_terrier,Staffordshire_bullterrier,,kelpie,0.537782
1377,684195085588783105,2016-01-05 02:09:54 +0000,Twitter for iPhone,This is Tino. He really likes corndogs. 9/10 h...,https://twitter.com/dog_rates/status/684195085...,9,10,Tino,,,...,6629631,132918,https://pbs.twimg.com/media/CX6_y6OU0AAl3v2.jpg,1,Chihuahua,toy_terrier,Boston_bull,,Chihuahua,0.379365
1376,684200372118904832,2016-01-05 02:30:55 +0000,Twitter for iPhone,Gang of fearless hoofed puppers here. Straight...,https://twitter.com/dog_rates/status/684200372...,6,10,None,,,...,6629628,132918,https://pbs.twimg.com/media/CX7EkuHWkAESLZk.jpg,1,llama,ram,hog,,Unidentifiable,0.000000
1375,684222868335505415,2016-01-05 04:00:18 +0000,Twitter for iPhone,Someone help the girl is being mugged. Several...,https://twitter.com/dog_rates/status/684222868...,121,110,None,,,...,6629628,132918,https://pbs.twimg.com/media/CX7Y_ByWwAEJdUy.jpg,1,soft-coated_wheaten_terrier,cocker_spaniel,teddy,,soft-coated_wheaten_terrier,0.791182
1374,684225744407494656,2016-01-05 04:11:44 +0000,Twitter for iPhone,"Two sneaky puppers were not initially seen, mo...",https://twitter.com/dog_rates/status/684225744...,143,130,None,,,...,6629628,132918,https://pbs.twimg.com/media/CX7br3HWsAAQ9L1.jpg,2,golden_retriever,Samoyed,Great_Pyrenees,,golden_retriever,0.203249


## _Define_ : ##
- Creating numerator and denominators in one column

## _Code_ : ##

In [77]:
# Creating rating df  
rating = twitter_archive_clean.text.str.extract('(\d+(\.\d+)*\/\d+)')

In [78]:
twitter_archive_clean['rating'] = rating[0]

## Test :

In [79]:
#looking the sampling distribution of df
twitter_archive_clean.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,...,user_favourites,jpg_url,img_num,p1,p2,p3,dog_stage,breed,confidence,rating
915,732726085725589504,2016-05-18 00:14:46 +0000,Twitter for iPhone,This is Aldrick. He looks wise af. Also except...,https://twitter.com/dog_rates/status/732726085...,11,10,Aldrick,,,...,132918,https://pbs.twimg.com/media/CisqdVcXEAE3iW7.jpg,1,Pomeranian,Samoyed,chow,,Pomeranian,0.961902,11/10
454,802239329049477120,2016-11-25 19:55:35 +0000,Twitter for iPhone,This is Loki. He'll do your taxes for you. Can...,https://twitter.com/dog_rates/status/802239329...,12,10,Loki,,,...,132918,https://pbs.twimg.com/media/CyIgaTEVEAA-9zS.jpg,2,Eskimo_dog,Siberian_husky,malamute,puppo,Eskimo_dog,0.482498,12/10
148,857029823797047296,2017-04-26 00:33:27 +0000,Twitter for iPhone,This is Zeke. He performs group cheeky wink tu...,https://twitter.com/dog_rates/status/857029823...,12,10,Zeke,,,...,132918,https://pbs.twimg.com/media/C-TIEwMW0AEjb55.jpg,2,golden_retriever,Labrador_retriever,Saluki,,golden_retriever,0.968623,12/10
294,828372645993398273,2017-02-05 22:40:03 +0000,Twitter for iPhone,This is Alexander Hamilpup. He was one of the ...,https://twitter.com/dog_rates/status/828372645...,12,10,Alexander,,,...,132918,https://pbs.twimg.com/media/C374hb0WQAAIbQ-.jpg,1,malamute,Eskimo_dog,Tibetan_mastiff,,malamute,0.663047,12/10
1330,687109925361856513,2016-01-13 03:12:26 +0000,Twitter for iPhone,This is Baxter. He looks like a fun dog. Prefe...,https://twitter.com/dog_rates/status/687109925...,11,10,Baxter,,,...,132918,https://pbs.twimg.com/media/CYka1NTWMAAOclP.jpg,2,borzoi,whippet,Saluki,,borzoi,0.883086,11/10


## _Define_ : ##
- Drop numerator and denominator columns

## _Code_ : ##

In [80]:
# inverting rating_numerator and rating_denominator to one column
twitter_archive_clean.drop(['rating_numerator', 'rating_denominator'], axis=1, inplace=True)

## Test :

In [81]:
# checking the changes
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 781
Data columns (total 23 columns):
tweet_id           1993 non-null int64
timestamp          1993 non-null object
source             1993 non-null object
text               1993 non-null object
expanded_urls      1993 non-null object
name               1361 non-null object
doggo              1993 non-null object
floofer            1993 non-null object
pupper             1993 non-null object
puppo              1993 non-null object
favorites          1993 non-null int64
retweets           1993 non-null int64
user_followers     1993 non-null int64
user_favourites    1993 non-null int64
jpg_url            1993 non-null object
img_num            1993 non-null int64
p1                 1993 non-null object
p2                 1993 non-null object
p3                 1993 non-null object
dog_stage          1993 non-null object
breed              1993 non-null object
confidence         1993 non-null float64
rating             

## _Define_ : ##
- Remove all tweets that don't include any image

## _Code_ : ##

In [82]:
#dropping all na in twitter_archive_clean
twitter_archive_clean = twitter_archive_clean.dropna(subset=['expanded_urls'])

## Test :

In [83]:
twitter_archive_clean['expanded_urls'].isnull().value_counts()

False    1993
Name: expanded_urls, dtype: int64

## _Define_ : ##
- Drop duplicate in jpg_url 

## _Code_ : ##

In [84]:
# dropping all 66 duplicate jpg_url
twitter_archive_clean['jpg_url'].drop_duplicates(keep = False, inplace = True)

## Test :

In [85]:
twitter_archive_clean[twitter_archive_clean['jpg_url'].duplicated(keep='first')]

,tweet_id,timestamp,source,text,expanded_urls,name,doggo,floofer,pupper,puppo,...,user_favourites,jpg_url,img_num,p1,p2,p3,dog_stage,breed,confidence,rating


In [86]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 781
Data columns (total 23 columns):
tweet_id           1993 non-null int64
timestamp          1993 non-null object
source             1993 non-null object
text               1993 non-null object
expanded_urls      1993 non-null object
name               1361 non-null object
doggo              1993 non-null object
floofer            1993 non-null object
pupper             1993 non-null object
puppo              1993 non-null object
favorites          1993 non-null int64
retweets           1993 non-null int64
user_followers     1993 non-null int64
user_favourites    1993 non-null int64
jpg_url            1993 non-null object
img_num            1993 non-null int64
p1                 1993 non-null object
p2                 1993 non-null object
p3                 1993 non-null object
dog_stage          1993 non-null object
breed              1993 non-null object
confidence         1993 non-null float64
rating             

## _Define_ : ##
- Extracting Dogs gender column from text column

## _Code_ : ##

In [87]:
# Loop on all the texts and check if it has one of pronouns of male or female
# and append the result in a list

male = ['He', 'he', 'him', 'his', "he's", 'himself']
female = ['She', 'she', 'her', 'hers', 'herself', "she's"]

dog_gender = []

for text in twitter_archive_clean['text']:
    # Male
    if any(map(lambda v:v in male, text.split())):
        dog_gender.append('male')
    # Female
    elif any(map(lambda v:v in female, text.split())):
        dog_gender.append('female')
    # If group or not specified
    else:
        dog_gender.append('NaN')

# Test
len(dog_gender)

# Save the result in a new column 'dog_name'
twitter_archive_clean['dog_gender'] = dog_gender

## **Test** ##

In [88]:
print("dog_gender count \n", twitter_archive_clean.dog_gender.value_counts())

dog_gender count 
 NaN       1131
male       636
female     226
Name: dog_gender, dtype: int64


## _Define_ : ##
- Convert the null values to None type

## _Code_ : ##

In [89]:
# converting null values to none
twitter_archive_clean.loc[twitter_archive_clean['dog_gender'] == 'NaN', 'dog_gender'] = None

## **Test** ##

In [90]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1993 entries, 0 to 781
Data columns (total 24 columns):
tweet_id           1993 non-null int64
timestamp          1993 non-null object
source             1993 non-null object
text               1993 non-null object
expanded_urls      1993 non-null object
name               1361 non-null object
doggo              1993 non-null object
floofer            1993 non-null object
pupper             1993 non-null object
puppo              1993 non-null object
favorites          1993 non-null int64
retweets           1993 non-null int64
user_followers     1993 non-null int64
user_favourites    1993 non-null int64
jpg_url            1993 non-null object
img_num            1993 non-null int64
p1                 1993 non-null object
p2                 1993 non-null object
p3                 1993 non-null object
dog_stage          1993 non-null object
breed              1993 non-null object
confidence         1993 non-null float64
rating             

## _Define_ : ##
- Change to right datatypes

## _Code_ : ##

In [89]:
# Changing to right dtypes
twitter_archive_clean['tweet_id'] = twitter_archive_clean['tweet_id'].astype(str)
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean.timestamp)
twitter_archive_clean['source'] = twitter_archive_clean['source'].astype('category')
twitter_archive_clean['favorites'] = twitter_archive_clean['favorites'].astype(int)
twitter_archive_clean['retweets'] = twitter_archive_clean['retweets'].astype(int)
twitter_archive_clean['dog_stage
twitter_archive_clean['user_followers'] = twitter_archive_clean['user_followers'].astype(int)
twitter_archive_clean['dog_gender'] = twitter_archive_clean['dog_gender'].astype('category')

In [90]:
# changing Unidentifiable bread to None
twitter_archive_clean.loc[twitter_archive_clean['breed'] == 'Unidentifiable', 'breed'] = None

## Test

In [91]:
twitter_archive_clean.dtypes

tweet_id                   object
timestamp          datetime64[ns]
source                   category
text                       object
expanded_urls              object
name                       object
doggo                      object
floofer                    object
pupper                     object
puppo                      object
favorites                   int64
retweets                    int64
user_followers              int64
user_favourites             int64
jpg_url                    object
img_num                     int64
p1                         object
p2                         object
p3                         object
stage                      object
breed                      object
confidence                float64
rating                     object
dog_gender               category
dtype: object

<a id='store'></a>
# **Store** #

In [92]:
# Save clean DataFrame to csv file
twitter_archive_clean.drop(twitter_archive_clean.columns[twitter_archive_clean.columns.str.contains('Unnamed',case = False)],axis = 1)
twitter_archive_clean.to_csv('twitter_df.csv', encoding = 'utf-8', index=False)